<a href="https://colab.research.google.com/github/shuuichi/GCP/blob/master/language_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


--- 
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```
---

## Google Cloud API の認証情報を入力

Google Cloud API を REST インタフェースから利用するために、 [API Key](https://console.cloud.google.com/apis/credentials) を利用します。 Google Cloud Console から API Key をコピーしましょう。

In [1]:
import getpass

APIKEY = getpass.getpass()

··········


## Cloud Natural Language API を使ってみよう !
**Cloud Natural Language API** は、エンティティ分析、感情分析、コンテンツ分類、構文解析などの自然言語を理解するための技術をデベロッパーに提供します。

API Discovery Service を利用して **Cloud Natural Language API** を発見します。 Cloud Natural Language の REST API 仕様は [こちら](https://cloud.google.com/natural-language/docs/reference/rest/) に解説されています。



In [0]:
from googleapiclient.discovery import build

nl_service = build('language', 'v1beta2', developerKey=APIKEY)

Cloud Natural Language API に入力する情報を定義します。

In [3]:
import textwrap

source_language = "en" #@param ["en", "ja"] {type: "string"}
source_sentence = "Classifying Opinion and Editorials can be time-consuming and difficult work for any data science team, but Cloud Natural Language was able to instantly identify clear topics with a high-level of confidence. This tool has saved me weeks, if not months, of work to achieve a level of accuracy that may not have been possible with our in-house resources." #@param {type:"string"}

document_type = "PLAIN_TEXT" #@param["PLAIN_TEXT", "HTML"] {type: "string"}
encoding_type = "UTF8" #@param["UTF8", "UTF16", "UTF32"] {type: "string"}

textwrap.wrap(source_sentence)

['Classifying Opinion and Editorials can be time-consuming and difficult',
 'work for any data science team, but Cloud Natural Language was able to',
 'instantly identify clear topics with a high-level of confidence. This',
 'tool has saved me weeks, if not months, of work to achieve a level of',
 'accuracy that may not have been possible with our in-house resources.']

### エンティティ分析

**エンティティ分析**は、指定されたテキストに既知のエンティティ（著名人、ランドマークなどの固有名詞）が含まれていないかどうかを調べて、それらのエンティティに関する情報を返します。エンティティ分析に関する REST API 仕様は[こちら](https://cloud.google.com/natural-language/docs/reference/rest/v1beta2/documents/analyzeEntities)で解説されています。

In [0]:
response = nl_service.documents().analyzeEntities(
  body={
    'document': {
      'type': document_type,
      'content': source_sentence,
      'language': source_language,
    },
    'encodingType': encoding_type,
  }
).execute()

In [5]:
# Below code is extracting only proper nouns from a response message. If you
# have interest on what information is exactly contained in a response message,
# please try to explore it :D

for entity in response['entities']:
  for mention in entity['mentions']:
    if mention['type'] == 'PROPER':
      print(mention, entity['metadata'])

{'text': {'content': 'Cloud Natural Language', 'beginOffset': 107}, 'type': 'PROPER'} {}


### 感情分析

**感情分析**は、指定されたテキストを調べて、そのテキストの背景にある感情的な考え方を分析します。具体的には、執筆者の考え方がポジティブか、ネガティブか、ニュートラルかを判断します。感情分析に関する REST API 仕様は[こちら](https://cloud.google.com/natural-language/docs/reference/rest/v1beta2/documents/analyzeSentiment)で解説されています。

In [0]:
response = nl_service.documents().analyzeSentiment(
  body={
    'document': {
      'type': document_type,
      'content': source_sentence,
      'language': source_language,
    },
    'encodingType': encoding_type,
  }
).execute()

In [14]:
# This shows you document-level sentiment.
response['documentSentiment']

{'magnitude': 1.6, 'score': 0.8}

In [15]:
# This shows you sentence-level sentiment.
for sentence in response['sentences']:
  print(sentence['sentiment'], sentence['text']['content'])

{'magnitude': 0.8, 'score': 0.8} Classifying Opinion and Editorials can be time-consuming and difficult work for any data science team, but Cloud Natural Language was able to instantly identify clear topics with a high-level of confidence.
{'magnitude': 0.7, 'score': 0.7} This tool has saved me weeks, if not months, of work to achieve a level of accuracy that may not have been possible with our in-house resources.


### コンテンツの分類 (英語のみ)

**コンテンツの分類**は、ドキュメントを分析し、ドキュメント内で見つかったテキストに適用されるコンテンツ カテゴリのリストを返します。ドキュメント内のコンテンツを分類するには、[classifyText](https://cloud.google.com/natural-language/docs/reference/rest/v1beta2/documents/classifyText) メソッドを呼び出します。



In [0]:
response = nl_service.documents().classifyText(
  body={
    'document': {
      'type': document_type,
      'content': source_sentence,
      'language': source_language,
    },
  }
).execute()

In [17]:
response['categories']

[{'confidence': 0.64, 'name': '/Computers & Electronics'}]

### 構文解析

**構文解析**では、指定されたテキストを一連の文とトークン（通常は単語）に分解して、それらのトークンに関する言語情報を提供します。ほとんどの Natural Language API メソッドは、指定されたテキストの内容を分析しますが、[analyzeSyntax](https://cloud.google.com/natural-language/docs/reference/rest/v1beta2/documents/analyzeSyntax)  メソッドは、言語自体の構造を検査します。

In [0]:
response = nl_service.documents().analyzeSyntax(
  body={
    'document': {
      'type': document_type,
      'content': source_sentence,
      'language': source_language,
    },
    'encodingType': encoding_type,
  }
).execute()

In [19]:
# This shows you output of syntax analysis. To leverage this output, you may
# need domain knowledge on natural language analysis.

response['tokens']

[{'dependencyEdge': {'headTokenIndex': 1, 'label': 'NN'},
  'lemma': 'Classifying',
  'partOfSpeech': {'aspect': 'ASPECT_UNKNOWN',
   'case': 'CASE_UNKNOWN',
   'form': 'FORM_UNKNOWN',
   'gender': 'GENDER_UNKNOWN',
   'mood': 'MOOD_UNKNOWN',
   'number': 'NUMBER_UNKNOWN',
   'person': 'PERSON_UNKNOWN',
   'proper': 'PROPER',
   'reciprocity': 'RECIPROCITY_UNKNOWN',
   'tag': 'NOUN',
   'tense': 'TENSE_UNKNOWN',
   'voice': 'VOICE_UNKNOWN'},
  'text': {'beginOffset': 0, 'content': 'Classifying'}},
 {'dependencyEdge': {'headTokenIndex': 5, 'label': 'NSUBJ'},
  'lemma': 'Opinion',
  'partOfSpeech': {'aspect': 'ASPECT_UNKNOWN',
   'case': 'CASE_UNKNOWN',
   'form': 'FORM_UNKNOWN',
   'gender': 'GENDER_UNKNOWN',
   'mood': 'MOOD_UNKNOWN',
   'number': 'SINGULAR',
   'person': 'PERSON_UNKNOWN',
   'proper': 'PROPER',
   'reciprocity': 'RECIPROCITY_UNKNOWN',
   'tag': 'NOUN',
   'tense': 'TENSE_UNKNOWN',
   'voice': 'VOICE_UNKNOWN'},
  'text': {'beginOffset': 12, 'content': 'Opinion'}},
 {'d

### 演習問題

** 1. エンティティ分析で著名人やランドマークを抽出してみましょう。**


** 2. 感情分析をさまざまな文章に対して実行してみましょう。 **

## Cloud Translation API を使ってみよう !

** Cloud Translation API** では、数千もの言語ペアの間でダイナミックにテキストを翻訳できます。 Cloud Translation API を使えば、プログラム上でウェブサイトやアプリケーションを Google Translate と統合できます。

API Discovery Service を利用して **Cloud Translation API** を発見します。 Cloud Natural Language の REST API 仕様は[こちら](https://cloud.google.com/translate/docs/reference/rest)で解説されています。

In [0]:
from googleapiclient.discovery import build

translate_service = build('translate', 'v2', developerKey=APIKEY)

Cloud Translation API に入力する情報を定義します。

In [50]:
import textwrap

source_language = "tr" #@param ["en", "ja"] {type: "string"}
target_language = "ja" #@param ["en", "ja"] {type: "string"}
source_sentence = "Google Cloud Translation API'nin en iyi ger\xE7ek zamanl\u0131 \xE7eviri sonu\xE7lar\u0131n\u0131 ve Google'\u0131n da en g\xFCvenilir hizmeti sundu\u011Funu g\xF6rd\xFCk. \xC7eviri ara\xE7lar\u0131 Clash of Kings sohbet hizmetiyle de sorunsuz \u015Fekilde entegre oldu." #@param {type:"string"}
textwrap.wrap(source_sentence, width=50)

["Google Cloud Translation API'nin en iyi gerçek",
 "zamanlı çeviri sonuçlarını ve Google'ın da en",
 'güvenilir hizmeti sunduğunu gördük. Çeviri',
 'araçları Clash of Kings sohbet hizmetiyle de',
 'sorunsuz şekilde entegre oldu.']

** 2. 次のテキストを英語に翻訳しましょう: **

Google Cloud Translation API'nin en iyi gerçek zamanlı çeviri sonuçlarını ve Google'ın da en güvenilir hizmeti sunduğunu gördük. Çeviri araçları Clash of Kings sohbet hizmetiyle de sorunsuz şekilde entegre oldu.

** 2. 次のテキストを英語に翻訳しましょう: **

Google Cloud Translation API'nin en iyi gerçek zamanlı çeviri sonuçlarını ve Google'ın da en güvenilir hizmeti sunduğunu gördük. Çeviri araçları Clash of Kings sohbet hizmetiyle de sorunsuz şekilde entegre oldu.

### テキストの翻訳

[translations](https://cloud.google.com/translate/docs/reference/translate) リクエストを使えば、入力テキストを、指定した言語に翻訳されたテキストを取得することが出来ます。入力テキストには**書式なしテキスト**または **HTML** を使用できます。Cloud Translation API は入力テキスト内の HTML タグは翻訳せず、タグ間にあるテキストのみを翻訳します。出力では、（未翻訳の）HTML タグは保持されます。タグは翻訳されたテキストに合わせて挿入されますが、ソース言語とターゲット言語には差異があるため、その位置調整は可能な範囲に留まります。出力内の HTML タグの順序は、翻訳による語順変化のため入力テキスト内の順序と異なる場合があります。

In [0]:
# Note that you can change Translation model by changing 'model' parameter.

response = translate_service.translations().list(
  source=source_language,
  target=target_language,
  q=source_sentence,
  model='nmt',
  format='html').execute()

In [46]:
text = response['translations'][0]['translatedText']
textwrap.wrap(text)

['意見や社説を分類することは、データサイエンスチームにとって時間がかかり困難な作業になる可能性がありますが、Cloud Natural Lan',
 'guageは、明確なトピックを高い信頼性で即座に特定することができました。このツールを使用すると、社内リソースでは不可能だったレベルの正確さ',
 'を達成するために、数か月間ではなくても数週間の作業が節約されました。']

### 言語の検出

[detections](https://cloud.google.com/translate/docs/reference/detect) リクエストを使えば、入力テキストの言語を特定できます。この機能は使われている言語が不明なテキストを自動的に翻訳するときなどに利用できます。


In [49]:
response = translate_service.detections().list(
  q=source_sentence
).execute()

response['detections']

[[{'confidence': 1, 'isReliable': False, 'language': 'tr'}]]

### 演習問題

In [29]:
print(source_language)
print(target_language)
print(source_sentence)

en
ja
Classifying Opinion and Editorials can be time-consuming and difficult work for any data science team, but Cloud Natural Language was able to instantly identify clear topics with a high-level of confidence. This tool has saved me weeks, if not months, of work to achieve a level of accuracy that may not have been possible with our in-house resources.


** 1. 翻訳モデルを NMT から PBT に変更するとどうなるか確かめましょう ** 

In [0]:
# Note that you can change Translation model by changing 'model' parameter.

response = translate_service.translations().list(
  source=source_language,
  target=target_language,
  q=source_sentence,
  model='base',
  format='html').execute()

In [37]:
text = response['translations'][0]['translatedText']
textwrap.wrap(text)

['分類意見や論説は、任意のデータサイエンスチームのために時間がかかり、困難な作業することができますが、クラウド自然言語は瞬時に自信の高レベルで',
 'クリアなトピックを識別することができました。このツールは、社内のリソースでは可能ではなかったかもしれない精度のレベルを達成するために仕事を、',
 'そうでない場合は数ヶ月、私は数週間を保存しています。']

** 2. 次のテキストを英語に翻訳しましょう: **

Google Cloud Translation API'nin en iyi gerçek zamanlı çeviri sonuçlarını ve Google'ın da en güvenilir hizmeti sunduğunu gördük. Çeviri araçları Clash of Kings sohbet hizmetiyle de sorunsuz şekilde entegre oldu.

In [51]:
import textwrap

source_language = "tr" #@param ["en", "ja"] {type: "string"}
source_sentence = "Google Cloud Translation API'nin en iyi gerçek zamanlı çeviri sonuçlarını ve Google'ın da en güvenilir hizmeti sunduğunu gördük. Çeviri araçları Clash of Kings sohbet hizmetiyle de sorunsuz şekilde entegre oldu." #@param {type:"string"}

document_type = "PLAIN_TEXT" #@param["PLAIN_TEXT", "HTML"] {type: "string"}
encoding_type = "UTF8" #@param["UTF8", "UTF16", "UTF32"] {type: "string"}

textwrap.wrap(source_sentence)

["Google Cloud Translation API'nin en iyi gerçek zamanlı çeviri",
 "sonuçlarını ve Google'ın da en güvenilir hizmeti sunduğunu gördük.",
 'Çeviri araçları Clash of Kings sohbet hizmetiyle de sorunsuz şekilde',
 'entegre oldu.']

In [0]:
# Note that you can change Translation model by changing 'model' parameter.

response = translate_service.translations().list(
  source=source_language,
  target=target_language,
  q=source_sentence,
  model='base',
  format='html').execute()

In [53]:
text = response['translations'][0]['translatedText']
textwrap.wrap(text)

['私たちは、Googleの検索結果にGoogleクラウド翻訳APIの最高のリアルタイム翻訳とは、最も信頼性の高いサービスを提供しているのを見ま',
 'した。キングスチャットサービスのクラッシュと翻訳ツールもシームレスに統合されました。']